# Distributed training with Keras example using MNIST dataset

This tutorial is adapted from the TensorFlow tutorial here: [https://www.tensorflow.org/tutorials/distribute/keras](https://www.tensorflow.org/tutorials/distribute/keras)

To run this on HiPerGator, I requested an [Open onDemand](https://ood.rc.ufl.edu/) Jupyter session with:
* 4 cores
* 64 GB RAM
* 2 A100 GPUs: `gpus:a100:2`

This tutorial is intended to help get you up and running training models on HiPerGator and to show how simple it can be to scale things up as you exceed the capabilities of a single GPU. This tutorial also introduced [TensorBoard](https://www.tensorflow.org/tensorboard/get_started#:~:text=TensorBoard%20is%20a%20tool%20for,dimensional%20space%2C%20and%20much%20more.), a relatively easy method of tracking model training and your hyperparameter tuning experiments.

## Load some modules and check TensorFlow version

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
print(tf.__version__)

2.7.0


## Load the MNIST dataset

Remember the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) is the handwritten image dataset that was featured in the NVIDIA Foundations of Deep Learning (or Getting Started with Deep Learning) course.

![Sample images from the MNIST dataset, Image by 
Suvanjanprasai from Wikipedia](images/MnistExamplesModified.png)

We don't really need multiple GPUs to analyze these data, but they are handy to use and a familiar example.

In [4]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

2023-08-27 19:27:46.354924: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 19:27:47.409554: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79111 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:b7:00.0, compute capability: 8.0
2023-08-27 19:27:47.412858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79111 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


## For multi-GPUs, setup a strategy

There are many different ways of using multiple GPUs in training a model. As with many things, without a framework, things can get complex quickly. Luckily Keras provides a nice API framework to do most of the work for us.

First, there are two main *things* that can be distributed across multiple GPUs:
* **Data parallelism** uses multiple GPUs to train a single model with each GPU evaluating different batches of the dataset and averaging results periodically.
* **Model parallelism** where a model is split across multiple GPUs. Models can be split with different layers on different GPUs, splitting the weights of a single layer across GPUs, or both.

As you scale beyond a single GPU, it is easier (and more efficient) to use multiple GPUs in a single server (8 in the DGX Servers), but after that, you need to use GPUs on multiple servers (hosts).

See more details [here](https://www.tensorflow.org/guide/keras/distributed_training).

The cool thing is, that setting up multi-GPU training, can be quite simple. In this case, we really only need a couple of lines:

In [5]:
# The MirroredStrategy works for multiple GPUs on 1 server.
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [6]:
# Double check how many GPUs our strategy sees
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


## Set up the input pipeline

As noted in the [original tutorial](https://www.tensorflow.org/tutorials/distribute/keras#set_up_the_input_pipeline), we can increase the batch size:
> When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory and tune the learning rate accordingly.

My guess is that on the A100s, we can go larger than 64, but let's start there.

In [7]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [8]:
def scale(image, label):
   '''Define a function that normalizes the image pixel values from the [0, 255] range to the [0, 1] range'''
   image = tf.cast(image, tf.float32)
   image /= 255

   return image, label

In [9]:
#Apply this scale function to the training and test data, 
# and then use the tf.data.Dataset APIs to shuffle the training 
# data (Dataset.shuffle), and batch it (Dataset.batch). 
# Notice that you are also keeping an in-memory cache of the training 
# data to improve performance (Dataset.cache).

train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## Create and compile the model

The main change here is that we make the model within the context of the `Strategy.scope`, but making and compiling the model is the same as it would be on a single GPU.

In [11]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Another interesting note in the tutorial about learning rate and using multiple GPUs:
> For this toy example with the MNIST dataset, you will be using the Adam optimizer's default learning rate of 0.001.
>
> For larger datasets, the key benefit of distributed training is to learn more in each training step, because each step processes more training data in parallel, which allows for a larger learning rate (within the limits of the model and dataset).

## Callbacks

Callbacks are the tools we can use to monitor training, control when to stop and lots of other useful features. We'll use some handy ones here.

In [15]:
# Define the checkpoint directory to store the checkpoints.
# This setus up logging for TensorBoard
checkpoint_dir = './training_checkpoints'
# Define the name of the checkpoint files.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [16]:
# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7: # Drop the learning rate for epochs 3-6
    return 1e-4
  else:                         # Drop the learning rate again for epochs 7 and on
    return 1e-5

In [17]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(        epoch + 1, model.optimizer.lr.numpy()))

In [18]:
# Put all the callbacks together.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [21]:
EPOCHS = 12

model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/12
  1/469 [..............................] - ETA: 14s - loss: 0.0269 - accuracy: 0.9922

2023-08-27 20:00:41.552129: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:537] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


462/469 [============================>.] - ETA: 0s - loss: 0.0421 - accuracy: 0.9870
Learning rate for epoch 1 is 0.0010000000474974513
469/469 [==============================] - 1s 3ms/step - loss: 0.0423 - accuracy: 0.9869 - lr: 0.0010
Epoch 2/12
464/469 [============================>.] - ETA: 0s - loss: 0.0299 - accuracy: 0.9908
Learning rate for epoch 2 is 0.0010000000474974513
469/469 [==============================] - 1s 3ms/step - loss: 0.0298 - accuracy: 0.9908 - lr: 0.0010
Epoch 3/12
465/469 [============================>.] - ETA: 0s - loss: 0.0224 - accuracy: 0.9929
Learning rate for epoch 3 is 0.0010000000474974513
469/469 [==============================] - 1s 3ms/step - loss: 0.0223 - accuracy: 0.9929 - lr: 0.0010
Epoch 4/12
465/469 [============================>.] - ETA: 0s - loss: 0.0111 - accuracy: 0.9977
Learning rate for epoch 4 is 9.999999747378752e-05
469/469 [==============================] - 1s 3ms/step - loss: 0.0111 - accuracy: 0.9977 - lr: 1.0000e-04
Epoch 5/12


In [22]:
# Check the checkpoint directory.
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00001
ckpt_10.data-00000-of-00001  ckpt_4.index
ckpt_10.index		     ckpt_5.data-00000-of-00001
ckpt_11.data-00000-of-00001  ckpt_5.index
ckpt_11.index		     ckpt_6.data-00000-of-00001
ckpt_12.data-00000-of-00001  ckpt_6.index
ckpt_12.index		     ckpt_7.data-00000-of-00001
ckpt_1.data-00000-of-00001   ckpt_7.index
ckpt_1.index		     ckpt_8.data-00000-of-00001
ckpt_2.data-00000-of-00001   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00001
ckpt_3.data-00000-of-00001   ckpt_9.index
ckpt_3.index


In [23]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))

24/79 [========>.....................] - ETA: 0s - loss: 0.0343 - accuracy: 0.9912 

2023-08-27 20:01:18.628843: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:537] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


79/79 [==============================] - 0s 2ms/step - loss: 0.0394 - accuracy: 0.9868
Eval loss: 0.03939875587821007, Eval accuracy: 0.9868000149726868


## Look at training in TensorBoard

Because of the way we are connected to the session with Jupyter, we can't easily use this session for running TensorBoard. Let's [follow the directions here to open a new HiPerGator Desktop](https://help.rc.ufl.edu/doc/Tensorboard) to look at the TensorBoard logs.

* In a new tab, go to [https://ood.rc.ufl.edu/](https://ood.rc.ufl.edu/)
* From the Interactive Apps menu, select the 1st option: HiPerGator Desktop
* Request:
   * 2 cores
   * 12 GB RAM
   * 1 hour of time
* Click Launch
* When the job starts, click the **Launch HiPerGator Desktop** button
* That will open a GUI Desktop in a new session.
* From the Applications menu, select **Terminal Emulator**
* Change directories to the directory where this notebook is located. e.g. `cd /blue/gms6029/$USER/distributed_mnist`
* Load the needed modules: `module load tensorflow ubuntu`
* Launch TensorBoard: `tensorboard --logdir=./logs &` 
    * The `&` tells bash to return rather than wait for the command to complete
* Wait a few seconds and once you see a line that says `TensorBoard 2.7.0 at http://localhost:6006/ (Press CTRL+C to quit)`, hit return to get the command prompt back
* Type: `firefox` to open the browser
* Go to `http://localhost:6006/`

